# Model Creator
## Creating the Final Result
### Tetrahedralization Awaits Those who are Patient

This notebook takes in a list of pixels (and the data associated with them) and turns those pixels into a model (or several models) of observation geometries on Titan. The procedure for this is rather simple: 

1) Add up I/F of all pixels in bins of incidece, emission, and azimuth angles. (5 degree separation in each bin).
2) Divide each bin by the number of pixels in said bin.
3) Tetrahedralize between each bin with pixels in it to interpolate to in-between bins that never had any pixels in them.
4) NEW: find the standard deviation for every bin.

With that, you have a completed model. 

The input for this notebook are "...PixelResults" files created by the PixelFinder. You are unlikely to be able to craft these files on your own without PixelFinder. 

The output of this notebook is either a single model, or several models each for a specific terrain type on Titan. 

The ModelCreator has the same options for sorting through data that PixelFinder does, so further subdivisions are allowed. This is not automatic, you will have to manually run each division for the model you want. Fortunately ModelCreator tends to be the fastest step (for larger data sets) so this won't be too much of a drain on resources.

Also you can choose if you want to examine a particular wavelength or just make models for all of them.

This first cell produces a model file out of simulation data. Simulation data from SRTC++ is done in C++ on FreeBSD elsewhere, the result it creates is a text file with everything already sorted into bins. All this really does is read the text file in and then save it as a .npy file.

In [24]:
#This first cell is for reading in simulation data. You don't have to run it for the rest of this notebook.

import csv
import math
import numpy as np

A = "A0.2"
wav = "5.00"

#Reading in a model...?
modelfilename = A + "iephi_consolidate_w"+wav+".Jcube.csv" #Insert file name for SIMULATION DATA here.
outfilename = "newDoose"+A+"LambSim"+wav+"um.npy" #what do you want the model to be called at the end?

X = 21
Y = 19
Z = 37
dataSim = [[[0 for x in range(Z)] for x in range(Y)] for x in range(X)] 
with open(modelfilename) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    i, j, k = 0, 0, 0
    for row in csv_reader:
        while (i < X):
            dataSim[i][j][k] = float(row[i])
            if (math.isnan(dataSim[i][j][k])):
                dataSim[i][j][k] = 0
            i = i + 1
        i = 0
        j = j + 1
        if (j >= Y):
            j = 0
            k = k + 1
        line_count += 1

np.save(outfilename, dataSim)

In [18]:
#PARAMETER SETTING
#This is where you tell the program what restrictions it should have on its observations.

inputFile = "equatorPixelResultsT.csv" #Shoudld be a "...PixelResults" file made by PixelFinder. Includes lots of pixel information.
inputFile = "..\\PixelFinder\\" + inputFile
outputName ="equatorDunesT" #DO NOT PUT A FILE EXTENSION! This name will be used as the basis for several files in some cases.
#IF YOU DO NOT CHANGE THAT IT WILL OVERRIDE THE FILE WITH THIS NAME

ignoreNull = True #Usually you're ignoring Null values on the mask. If you're looking at a specific area and don't care, set this to False.

#Set values you want to manage to True
#CAREFUL! Setting more than a couple options to True will produce a LOT of models. 

terrainManage = False #If True, ModelCreator will create a model for each terrain type. (Except Null, unless ignoreNull is False).
allWindows = True #Set to True if you want to make models for every atmospheric IR window.
terrainSpec = "Dunes" #If terrainManage is False, still specifically select only this terrain. Set to "Null" if you don't care. (Since you should never want to single out Null terrain values...)
windowSpec = 5 #1-8 for selecting a window. 5 is 2um, the usual result. 

#The rest of these are usually False unless you're subdividing a model for some reason.

latLonCheck = False
flybyCheck = False #Temporal restriction by flyby number
resolutionCheck = False 

maskDistCheck = False
maskDistResRelCheck = False

#Latlon
latUpperBound = 30.0
latLowerBound = -30.0
lonUpperBound = 360.0
lonLowerBound = 0.0

#Flyby
flybyUpperBound = 100.0
flybyLowerBound = 1.0
#For TA and TB use 1 and 2.

#Resolution
resUpperBound = 25.0 #25.0 is considered standard. 
resLowerBound = 0.0

#Mask Stuff
#maskDistcheck
maskDistLimit = 50.0
#Only distances greater than this (km) will be accepted.

#maskDistresRelCheck
maskDistRelLimit = 1.0/4.0 # 1/4 is the standard one, meaning the distance to an untrusted pixel must be four times the pixel's resolution to pass judgment. 

#Things that are NOT sorted here:
#Cube coordinates, since there's no reason to sift via those, they just help you locate the pixel again later on. 
#Viewing Geometry Angles. The models we create want the largest range of these you can obtain for proper tetrahedralization.
#I/F of various windows, as restricting these values will create inacurate models at the end. 
#Cube number, as the flyby number gives enough of a time judgment for our purposes. 

In [19]:
import csv
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.animation as animation

#set up the things that will hold it all.
X = 21
Y = 19
Z = 37

W = 1 #Default, only one wavelength
T = 1 #Default, no terrain distinctions

select = windowSpec + 12 #Value we select for. 17 should be 2um.

#So here comes a somewhat complciated part. Each individual model is a 3D matrix, but we need to make one of these for each wavelength and terrain combination we care about. So we need a 5D matrix. Which is going to be wild.
if (terrainManage == True):
    T = 8 #There are 8 terrain codes in the mask due to how it is color coded. Null, Plains, Dunes, Xanadu, Hummocks, Lakes, Labyrinths, Craters.
if (allWindows == True): 
    W = 8 #There are 8 different atmospheric windows.

allModels = [[[[[0 for x in range(Z)] for x in range(Y)] for x in range(X)] for x in range(T)] for x in range (W)]
allModelsNum = [[[[[0 for x in range(Z)] for x in range(Y)] for x in range(X)] for x in range(T)] for x in range (W)]
allModelsStdev = [[[[[0 for x in range(Z)] for x in range(Y)] for x in range(X)] for x in range(T)] for x in range (W)]

#allModels holds everything. The indeces go window, terrain, incidence, emission, azimuth. 
#allModelsNum holds the number of times each bin has been hit, so we can average later.

#This was added later: we had some desire to know where the data came from. Instead of a number, each entry will be a flyby/cube pair.
allModelsCubeList = [[[[[[] for x in range(Z)] for x in range(Y)] for x in range(X)] for x in range(T)] for x in range (W)]

#Read the data in, counting how many times we hit each bin.

flybyRangeAcceptable = False

with open(inputFile) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    line_count = 0
    for row in csv_reader:
        inci = float(row[7])
        emis = float(row[8])
        if (float(row[9]) > 200.):
            azim = math.radians(float(row[9]))
        else:
            azim = float(row[9])
        #Our goal is to convert these to indices for every "5" degrees.
        #So, divide by 5 then round to get the index. 

        #Get the flyby and cube number.
        cubeID = [row[0],row[1]]

        passedChecks = True #Unless something trips, this remains true.
        
        if (latLonCheck == True):
            if ((float(row[5]) > latLowerBound and float(row[5]) < latUpperBound and float(row[6]) > lonLowerBound and float(row[6]) < lonUpperBound) == False):
                passedChecks = False

        flybyRangeAcceptable = False
        flyby = 0.
        if (row[0] == "TA"): flyby = 1.
        elif (row[0] == "TB"): flyby = 2.
        else: flyby = float(row[0][1:])
        if (flybyCheck == True): #Are we in the right flyby range?
            if (flybyLowerBound <= flyby and flybyUpperBound >= flyby):
                flybyRangeAcceptable = True
        else: 
            flybyRangeAcceptable = True #If we're not checking for time, all of them are fine. 
        if (flybyRangeAcceptable == False):
            passedChecks = False

        if (resolutionCheck == True):
            if ( (resUpperBound > float(row[10]) and resLowerBound < float(row[10]) and float(row[10]) != 0.0)== False ):
                passedChecks = False

        if (maskDistCheck == True):
            if (float(row[11]) < maskDistLimit):
                passedChecks = False

        if (maskDistResRelCheck == True):
            if ( float(row[10]) > maskDistRelLimit*float(row[11]) ):
                passedChecks = False     

        if (passedChecks == True):
            if (allWindows == False):
                if (terrainManage == True):
                    if (row[2] == "Null" and ignoreNull == False): #Only catalog Nulls if desired.
                        allModelsNum[0][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModelsNum[0][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + 1
                        allModels[0][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModels[0][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + float(row[select])
                        allModelsCubeList[0][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))].append(cubeID)
                    elif (row[2] == "Lake"):
                        allModelsNum[0][1][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModelsNum[0][1][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + 1
                        allModels[0][1][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModels[0][1][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + float(row[select])
                        allModelsCubeList[0][1][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))].append(cubeID)
                    elif (row[2] == "Xanadu"):
                        allModelsNum[0][2][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModelsNum[0][2][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + 1
                        allModels[0][2][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModels[0][2][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + float(row[select])
                        allModelsCubeList[0][2][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))].append(cubeID)
                    elif (row[2] == "Crater"):
                        allModelsNum[0][3][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModelsNum[0][3][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + 1
                        allModels[0][3][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModels[0][3][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + float(row[select])
                        allModelsCubeList[0][3][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))].append(cubeID)
                    elif (row[2] == "Dunes"):
                        allModelsNum[0][4][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModelsNum[0][4][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + 1
                        allModels[0][4][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModels[0][4][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + float(row[select])
                        allModelsCubeList[0][4][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))].append(cubeID)
                    elif (row[2] == "Labyrinth"):
                        allModelsNum[0][5][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModelsNum[0][5][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + 1
                        allModels[0][5][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModels[0][5][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + float(row[select])
                        allModelsCubeList[0][5][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))].append(cubeID)
                    elif (row[2] == "Hummocky"):
                        allModelsNum[0][6][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModelsNum[0][6][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + 1
                        allModels[0][6][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModels[0][6][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + float(row[select])
                        allModelsCubeList[0][6][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))].append(cubeID)
                    elif (row[2] == "Plains"):
                        allModelsNum[0][7][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModelsNum[0][7][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + 1
                        allModels[0][7][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModels[0][7][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + float(row[select])
                        allModelsCubeList[0][7][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))].append(cubeID)
                else: #This is what happens if you aren't separating terrain or wavelengths.
                    if (terrainSpec == "Null"):
                        if (row[2] != "Null" or ignoreNull == False): #Nulls can still be ignored, of course, if desired.
                            allModelsNum[0][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModelsNum[0][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + 1
                            allModels[0][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModels[0][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + float(row[select])
                            allModelsCubeList[0][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))].append(cubeID)
                    else:
                        if (row[2] == terrainSpec): #If you just want one terrain type.
                            allModelsNum[0][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModelsNum[0][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + 1
                            allModels[0][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModels[0][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + float(row[select])
                            allModelsCubeList[0][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))].append(cubeID)
                        
                        
            else:
                i = 0
                while (i < W):
                    if (terrainManage == True):
                        if (row[2] == "Null" and ignoreNull == False):
                            allModelsNum[i][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModelsNum[i][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + 1
                            allModels[i][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModels[i][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + float(row[13+i])
                            allModelsCubeList[i][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))].append(cubeID)
                        elif (row[2] == "Lake"):
                            allModelsNum[i][1][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModelsNum[i][1][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + 1
                            allModels[i][1][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))]= allModels[i][1][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + float(row[13+i])
                            allModelsCubeList[i][1][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))].append(cubeID)
                        elif (row[2] == "Xanadu"):
                            allModelsNum[i][2][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModelsNum[i][2][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + 1
                            allModels[i][2][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModels[i][2][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + float(row[13+i])
                            allModelsCubeList[i][2][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))].append(cubeID)
                        elif (row[2] == "Crater"):
                            allModelsNum[i][3][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModelsNum[i][3][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + 1
                            allModels[i][3][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModels[i][3][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + float(row[13+i])
                            allModelsCubeList[i][3][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))].append(cubeID)
                        elif (row[2] == "Dunes"):
                            allModelsNum[i][4][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModelsNum[i][4][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + 1
                            allModels[i][4][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModels[i][4][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + float(row[13+i])
                            allModelsCubeList[i][4][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))].append(cubeID)
                        elif (row[2] == "Labyrinth"):
                            allModelsNum[i][5][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModelsNum[i][5][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + 1
                            allModels[i][5][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModels[i][5][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + float(row[13+i])
                            allModelsCubeList[i][5][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))].append(cubeID)
                        elif (row[2] == "Hummocky"):
                            allModelsNum[i][6][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModelsNum[i][6][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + 1
                            allModels[i][6][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModels[i][6][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))]+ float(row[13+i])
                            allModelsCubeList[i][6][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))].append(cubeID)
                        elif (row[2] == "Plains"):
                            allModelsNum[i][7][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModelsNum[i][7][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + 1
                            allModels[i][7][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModels[i][7][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + float(row[13+i])
                            allModelsCubeList[i][7][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))].append(cubeID)
                    else: 
                        #This is what happens if you are separating wavelengths but not terrain.
                        if (terrainSpec == "Null"):
                            if (row[2] != "Null" or ignoreNull == False): #Nulls can still be ignored, of course, if desired.
                                allModelsNum[i][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModelsNum[i][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + 1
                                allModels[i][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModels[i][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + float(row[13+i])
                                allModelsCubeList[i][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))].append(cubeID)
                        else:
                            if (row[2] == terrainSpec): #If you just want one specific terrain type.
                                allModelsNum[i][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModelsNum[i][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + 1
                                allModels[i][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] = allModels[i][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))] + float(row[13+i])
                                allModelsCubeList[i][0][int(np.round(inci/5.))][int(np.round(emis/5.))][int(np.round(azim/5.))].append(cubeID)
                    i=i+1
        line_count += 1

i,j,k,l,m = 0,0,0,0,0
while (i < W):
    j = 0
    while (j < T):
        k = 0
        while (k < X):
            l = 0
            while (l < Y):
                m = 0
                while (m < Z):
                    if (allModelsNum[i][j][k][l][m] != 0):
                        allModels[i][j][k][l][m] = allModels[i][j][k][l][m] / float(allModelsNum[i][j][k][l][m])
                        #Divide out the total additions so we get an average value.
                    m=m+1
                l=l+1
            k=k+1
        j=j+1
    i=i+1

print(allModelsCubeList[0])
#print(allModelsNum)

[[[[[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [['T37', '1574127168_1'], ['T37', '1574127168_1'], ['T37', '1574127168_1'], ['T37', '1574127168_1'], ['T37', '1574127168_1'], ['T37', '1574127168_1'], ['T37', '1574127168_1']], [['T37', '1574127168_1'], ['T37', '1574127168_1'], ['T37', '1574127168_1'], ['T37', '1574127168_1']]], [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []], [[['T72', '1664052118_1'], ['T72', '1664052118_1'], ['T72', '1664052118_1'], ['T72', '1664052118_1']], [['T72', '1664052118_1'

In [20]:
#ALERT: DOES NOT DIFFERENTIATE TERRAIN TYPES AT THIS TIME BE CAREFUL!!!

#REFORMATTED WITH GEMINI.

import csv
import math
import numpy as np

# --- CONFIGURATION ---
# Assuming allModelsStdev, allModelsNum, and allModelsCubeList are already defined as per your snippet

# --- STEP 1: READ FILE ONCE AND CACHE DATA ---
print("Reading and caching file...")

# Cache Structure: 
# dictionary[(k, l, m)] -> { (cubeID_0, cubeID_1): [ [list_of_i_values], [list_of_i_values] ] }
pixel_cache = {}

with open(inputFile, 'r') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    
    for row in csv_reader:
        # 1. Filter Check
        #if row[2] != "Dunes": 
            #continue

        # 2. Geometry Calculations
        # Note: Added try/except to handle potential bad data/headers safely
        try:
            inci = float(row[7])
            emis = float(row[8])
            raw_azim = float(row[9])
            
            azim = math.radians(raw_azim) if raw_azim > 200. else raw_azim
            
            # Calculate Indices
            k_idx = int(np.round(inci/5.))
            l_idx = int(np.round(emis/5.))
            m_idx = int(np.round(azim/5.))
            
            # 3. Store in Cache
            # We use a tuple for the dictionary key because it's hashable
            bin_key = (k_idx, l_idx, m_idx)
            cube_id = (row[0], row[1]) # Tuple is safer than list for dict keys
            
            if bin_key not in pixel_cache:
                pixel_cache[bin_key] = {}
            if cube_id not in pixel_cache[bin_key]:
                pixel_cache[bin_key][cube_id] = []
            
            # Optimization: Read ALL 'i' values (spectral data) at once.
            # Assuming data starts at col 13 and goes for length W. 
            # We store the slice so we don't have to re-read per 'i' loop.
            spectral_data = [float(x) for x in row[13 : 13 + W]]
            pixel_cache[bin_key][cube_id].append(spectral_data)
            
        except (ValueError, IndexError):
            continue # Skip malformed rows

print("File cached. Starting calculation...")

# --- STEP 2: CALCULATE STANDARD DEVIATION ---

i,j,k,l,m = 0,0,0,0,0

while (i < W):
    print(f"Processing Band i={i}") # Moved print here to be less spammy but informative
    j = 0
    while (j < T):
        k = 0
        while (k < X):
            l = 0
            while (l < Y):
                m = 0
                while (m < Z):
                    # Only process if we expect data here
                    if (allModelsNum[i][j][k][l][m] != 0):
                        
                        # 1. Retrieve the list of valid cubes for this bin
                        # Convert to tuples to match our cache keys
                        valid_cubes_list = allModelsCubeList[i][j][k][l][m]
                        # Assuming structure is [[id1, id2], [id3, id4]] -> convert to set of tuples for O(1) lookup
                        valid_cubes_set = set(tuple(c) for c in valid_cubes_list)
                        
                        cubeCount = allModelsNum[i][j][k][l][m]
                        windowVals = []
                        
                        # 2. Look up data in our Memory Cache
                        # Do we have data for this geometry (k,l,m)?
                        if (k, l, m) in pixel_cache:
                            # Iterate through the cubes we found in the file for this geometry
                            found_cubes_dict = pixel_cache[(k, l, m)]
                            
                            for file_cube_id, data_rows in found_cubes_dict.items():
                                # 3. Intersection: Is the file's cube in our 'valid cubes' list?
                                if file_cube_id in valid_cubes_set:
                                    # Collect the specific 'i' value from the cached rows
                                    for row_values in data_rows:
                                        # Ensure we don't go out of bounds if W matches row length
                                        if i < len(row_values):
                                            windowVals.append(row_values[i])


                        # 4. Math
                        if len(windowVals) > 0:
                            # Use the actual count of found pixels or the pre-calculated N?
                            # Using len(windowVals) is usually safer, but sticking to your logic:
                            # Note: np.std defaults to population std. Use ddof=1 for sample std if needed.
                            allModelsStdev[i][j][k][l][m] = np.std(windowVals)
                        else:
                            # If allModelsNum was > 0 but we found no matching pixels in file
                            # Keep as default or set to 0? Keeping default.
                            pass

                    m += 1
                l += 1
            k += 1
        j += 1
    i += 1

print("Calculation Complete.")
print(allModelsStdev[0]) # Test print

#SAVING
#MMmmstandardDeviation. 
windowum = ["0.93", "1.08", "1.27", "1.59", "2.01", "2.69", "2.79", "5.00"]
terraintype = ["Null", "Lake", "Xanadu", "Crater", "Dunes", "Labyrinth", "Hummocky", "Plains"]

i,j = 0,0
while (i < W):
    j = 0
    while (j < T):

        goodArrayTest = False #Do not write empty arrays, that'll just take up space.
        for item in allModelsStdev[i][j]:
            for subitem in item:
                for subsubitem in subitem:
                    if (subsubitem != 0):
                        goodArrayTest = True

        if (goodArrayTest == True):
            if (T == 1):
                if (W == 1):
                    np.save(outputName + windowum[windowSpec - 1] + "umStdev" + ".npy", allModelsStdev[i][j])
                else:
                    np.save(outputName + windowum[i] + "umStdev" + ".npy", allModelsStdev[i][j])
            else:
                if (W == 1):
                    np.save(outputName + windowum[windowSpec - 1] + "umStdev" + terraintype[j] + ".npy", allModelsStdev[i][j])
                else:
                    np.save(outputName + windowum[i] + "umStdev" + terraintype[j] + ".npy", allModelsStdev[i][j])
        j=j+1
    i=i+1

Reading and caching file...
File cached. Starting calculation...
Processing Band i=0
Processing Band i=1
Processing Band i=2
Processing Band i=3
Processing Band i=4
Processing Band i=5
Processing Band i=6
Processing Band i=7
Calculation Complete.
[[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, np.float64(0.0016634463978040756), np.float64(0.003839399464760077)], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [np.float64(0.003317743915298467), np.float64(0.0), np.float64(0.0034544659175493675), np.float64(0.004423870577773378), np.float64(0.0029548936000303025), np.float64(0.0033873435388221255), np.float64(0.0006464999999999943), np.float6

In [4]:
#TETRAHEDRALIZATION STEP

#IMPORTS
import math
import pyvista as pv

#EXTRACT: Identify where the actual point values are in the matrix, and put them in an array.
pointArray = [[[] for x in range(T)] for x in range(W)]
i,j,k,l,m = 0,0,0,0,0
while (i < W):
    j = 0
    while (j < T):
        k = 0
        while (k < X):
            l = 0
            while (l < Y):
                m = 0
                while (m < Z):
                    if (allModels[i][j][k][l][m] != 0):
                        pointArray[i][j].append([k,l,m])
                    m=m+1
                l=l+1
            k=k+1
        j=j+1
    i=i+1
#The pointarray has now identified every nonzero point.

tetrahedralizedModels = [[[[[0 for x in range(Z)] for x in range(Y)] for x in range(X)] for x in range(T)] for x in range (W)]
#For storing the actual results.

i,j,k,l,m = 0,0,0,0,0
#Have to be careful in this loop with the various objects we create.
while (i < W):
    j = 0
    while (j < T):
        k = 0

        #We need to be very careful never to tetrahedralize an empty array, which may happen due to, say, not having any Craters to place into the Crater model. 
        zeroTest = False
        print(i,j)
        while(zeroTest == False):
            if (len(pointArray[i][j]) < 4): #Three points might as well be an empty array since we can't tetrahedralize that.
                j=j+1
                if (j == T):
                    i=i+1
                    j=0
                    if (i==W):
                        j=T
                        k=X #Make sure we don't try to read it.
                        zeroTest = True #Technically speaking, this is a failure mode, but we have to break out of the loop at the last iteration anyway.
            else:
                zeroTest = True
                #Only if there are enough points in the point array.

        #For every model, we need a mesh.
        if (i == W and j == 8): break
        print(i,j)
        mesh = pv.PolyData(pointArray[i][j]).delaunay_3d() 
        
        while (k < X):
            l = 0
            while (l < Y):
                m = 0
                while (m < Z):
                    if (allModels[i][j][k][l][m] != 0):
                        tetrahedralizedModels[i][j][k][l][m] = allModels[i][j][k][l][m]
                        #If we are on a point, don't bother interpolating. 
                    else:
                        cellNum = mesh.find_containing_cell([k,l,m])
                        if (cellNum == -1):
                            tetrahedralizedModels[i][j][k][l][m] = -1
                            #Exterior Values get Zero'd
                        else:
                            cell = mesh.get_cell(cellNum)
                            cellPoints = cell.points
                            #And now we can do the actual interpolation.
                            pointVals = []
                            iter = 0
                            while (iter < 4):
                                pointVals.append(allModels[i][j][int(cellPoints[iter][0])][int(cellPoints[iter][1])][int(cellPoints[iter][2])])
                                iter=iter+1
                            #pointVals hold the values at all points.
                            #Now the matrix magic is below
                            M = [[cellPoints[0][0],cellPoints[0][1],cellPoints[0][2],1.0],
                                [cellPoints[1][0],cellPoints[1][1],cellPoints[1][2],1.0],
                                [cellPoints[2][0],cellPoints[2][1],cellPoints[2][2],1.0],
                                [cellPoints[3][0],cellPoints[3][1],cellPoints[3][2],1.0]]
                            V = (1./6.)*np.absolute(np.linalg.det(M))
                            M1 = [[float(k),float(l),float(m),1.0],
                                [cellPoints[1][0],cellPoints[1][1],cellPoints[1][2],1.0],
                                [cellPoints[2][0],cellPoints[2][1],cellPoints[2][2],1.0],
                                [cellPoints[3][0],cellPoints[3][1],cellPoints[3][2],1.0]]
                            V1 = (1./6.)*np.absolute(np.linalg.det(M1))
                            M2 = [[cellPoints[0][0],cellPoints[0][1],cellPoints[0][2],1.0],
                                [float(k),float(l),float(m),1.0],
                                [cellPoints[2][0],cellPoints[2][1],cellPoints[2][2],1.0],
                                [cellPoints[3][0],cellPoints[3][1],cellPoints[3][2],1.0]]
                            V2 = (1./6.)*np.absolute(np.linalg.det(M2))
                            M3 = [[cellPoints[0][0],cellPoints[0][1],cellPoints[0][2],1.0],
                                [cellPoints[1][0],cellPoints[1][1],cellPoints[1][2],1.0],
                                [float(k),float(l),float(m),1.0],
                                [cellPoints[3][0],cellPoints[3][1],cellPoints[3][2],1.0]]
                            V3 = (1./6.)*np.absolute(np.linalg.det(M3))
                            M4 = [[cellPoints[0][0],cellPoints[0][1],cellPoints[0][2],1.0],
                                [cellPoints[1][0],cellPoints[1][1],cellPoints[1][2],1.0],
                                [cellPoints[2][0],cellPoints[2][1],cellPoints[2][2],1.0],
                                [float(k),float(l),float(m),1.0]]
                            V4 = (1./6.)*np.absolute(np.linalg.det(M4))
                            #print(V, V1, V2, V3, V4, V1+V2+V3+V4)
                            #The volumes of each of these segments correspond to the vertex not part of their calculation.
                            #Thus we interpolate with simple divisions:
                            interpVal = pointVals[0]*(V1/V) + pointVals[1]*(V2/V) + pointVals[2]*(V3/V) + pointVals[3]*(V4/V)
                            tetrahedralizedModels[i][j][k][l][m] = float(interpVal) 
                    m=m+1
                l=l+1
            k=k+1
        j=j+1
    i=i+1

#print(tetrahedralizedModels)

0 0
0 0


\\?\C:\Users\deran\AppData\Roaming\jupyterlab-desktop\jlab_server\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(


1 0
1 0
2 0
2 0
3 0
3 0
4 0
4 0
5 0
5 0
6 0
6 0
7 0
7 0


In [5]:
#SAVING
#Get your models saved nice and proper like.
windowum = ["0.93", "1.08", "1.27", "1.59", "2.01", "2.69", "2.79", "5.00"]
terraintype = ["Null", "Lake", "Xanadu", "Crater", "Dunes", "Labyrinth", "Hummocky", "Plains"]

i,j = 0,0
while (i < W):
    j = 0
    while (j < T):

        goodArrayTest = False #Do not write empty arrays, that'll just take up space.
        for item in tetrahedralizedModels[i][j]:
            for subitem in item:
                for subsubitem in subitem:
                    if (subsubitem != 0):
                        goodArrayTest = True

        if (goodArrayTest == True):
            if (T == 1):
                if (W == 1):
                    np.save(outputName + windowum[windowSpec - 1] + "um" + ".npy", tetrahedralizedModels[i][j])
                else:
                    np.save(outputName + windowum[i] + "um" + ".npy", tetrahedralizedModels[i][j])
            else:
                if (W == 1):
                    np.save(outputName + windowum[windowSpec - 1] + "um" + terraintype[j] + ".npy", tetrahedralizedModels[i][j])
                else:
                    np.save(outputName + windowum[i] + "um" + terraintype[j] + ".npy", tetrahedralizedModels[i][j])
        j=j+1
    i=i+1

In [6]:
#In a separate block so I can save without retetrahedralizing everything. 

windowum = ["0.93", "1.08", "1.27", "1.59", "2.01", "2.69", "2.79", "5.00"]
terraintype = ["Null", "Lake", "Xanadu", "Crater", "Dunes", "Labyrinth", "Hummocky", "Plains"]

# ALSO save the counts, which we can use to identify which pixels are "real"

i,j = 0,0
while (i < W):
    j = 0
    while (j < T):

        goodArrayTest = False #Do not write empty arrays, that'll just take up space.
        for item in allModelsNum[i][j]:
            for subitem in item:
                for subsubitem in subitem:
                    if (subsubitem != 0):
                        goodArrayTest = True

        if (goodArrayTest == True):
            if (T == 1):
                if (W == 1):
                    np.save(outputName + windowum[windowSpec - 1] + "um" + "Counts.npy", allModelsNum[i][j])
                else:
                    np.save(outputName + windowum[i] + "um" + "Counts.npy", allModelsNum[i][j])
            else:
                if (W == 1):
                    np.save(outputName + windowum[windowSpec - 1] + "um" + terraintype[j] + "Counts.npy", allModelsNum[i][j])
                else:
                    np.save(outputName + windowum[i] + "um" + terraintype[j] + "Counts.npy", allModelsNum[i][j])
        j=j+1
    i=i+1

#NOTE: This next one is saved with PICKLE, so to read it you have to allow PICKLE when loading. Don't forget that. Even though it says pickle nowhere here.
i,j = 0,0
while (i < W):
    j = 0
    while (j < T):

        goodArrayTest = False #Do not write empty arrays, that'll just take up space.
        for item in allModelsCubeList[i][j]:
            for subitem in item:
                for subsubitem in subitem:
                    if (subsubitem != []):
                        goodArrayTest = True

        if (goodArrayTest == True):
            if (T == 1):
                if (W == 1):
                    dataToSave = np.array(allModelsCubeList[i][j], dtype=object)
                    np.save(outputName + windowum[windowSpec - 1] + "um" + "Cubs.npy", dataToSave)
                else:
                    dataToSave = np.array(allModelsCubeList[i][j], dtype=object)
                    np.save(outputName + windowum[i] + "um" + "Cubs.npy", dataToSave)
            else:
                if (W == 1):
                    dataToSave = np.array(allModelsCubeList[i][j], dtype=object)
                    np.save(outputName + windowum[windowSpec - 1] + "um" + terraintype[j] + "Cubs.npy", dataToSave)
                else:
                    dataToSave = np.array(allModelsCubeList[i][j], dtype=object)
                    np.save(outputName + windowum[i] + "um" + terraintype[j] + "Cubs.npy", dataToSave)
        j=j+1
    i=i+1


# Quick and Dirty Model Displayer

## You do have to manually add the name of the model you just made though.

### Only displays one at a time, sorry batch model creators.

Note: make sure to use the animation ending cell below this one before trying to run again, otherwise you may end up with overlapping animations.

In [25]:
#REMEMBER TO END ANIMATION BEFORE RUNNING CELL TWICE
%matplotlib ipympl

#Values for you to set
displayModel = np.load("refinementDunes2um200dist2.01um.npy")
#The scale value for how much color detail we want:
cmap = 'viridis'
vmin=0
vmax=0.5

#Plotting Imports
import matplotlib.pyplot as plt
import matplotlib.animation as animation

fig, ax = plt.subplots() #matplotlib's axis and figure notation is so annoying and obtuse...

#Now we'd like to try to animate it. 
#https://matplotlib.org/stable/users/explain/animations/animations.html
#Eventually had to use ChatGPT to help here. Could not get the ArtistAnimator to work, but it could
#get the FuncAnimation to work--evidentially defining zz inside the funciton/loop was important.
#Even though I thought I already tried that...

#Now we actually need to get the slices
#Set up x and y divisions.        
xx = np.linspace(0, 100, 21)
yy = np.linspace(0, 90, 19)

c = ax.pcolor(xx,yy, np.zeros((Y, X)), cmap=cmap,vmin=vmin,vmax=vmax)
fig.colorbar(c, ax=ax, label='I/F Average')
# annoyance of animation: trying to update the color bar duplicates it.
# so we need a "dummy" plot to initialize it before we start animating. 

#Making an array so the title can display the azimuth angle. 
angles = ["000", "005", "010", "015", "020", "025", "030", "035", "040", "045", "050", "055", "060", "065", "070", "075",
             "080", "085", "090", "095", "100", "105", "110", "115", "120", "125", "130", "135", "140", "145", "150", "155",
             "160", "165", "170", "175", "180"] 

def update(frame):
    ax.clear()
    zz = np.zeros((Y, X))
    for x in range(X):
        for y in range(Y):
            zz[y][x] = displayModel[x][y][frame]
    azimuth = ax.pcolor(xx, yy, zz, cmap=cmap,vmin=vmin,vmax=vmax)
    ax.set(xlim=[-2.5,102.5], ylim=[-2.5,92.5], xlabel='Incidence Angle', ylabel='Emission Angle', 
           title = 'Test Model. Azimuth: '+str(angles[frame])+ ' degrees')
    

ani = animation.FuncAnimation(fig, update, frames=Z, interval=500)

# To save the animation using Pillow as a gif. Use FFMpegFileWriter for mp4. 
#writer = animation.FFMpegFileWriter(fps = 2)
#ani.save('TEST2.mp4', writer = writer)
#writer = animation.PillowWriter(fps=5)
#ani.save('TEST2.gif', writer=writer)

plt.show()

FileNotFoundError: [Errno 2] No such file or directory: 'refinementDunes2um200dist2.01um.npy'

In [ ]:
#always end the animation before running it again.

ani.pause()
plt.close()

In [4]:
#A simple sifting algorithm to snip away the file without all the complexity.
#Was used to reduce a pixelresults file to just Dunes.
#Maybe useful elsehwere one day, so not thrown away.
#https://stackoverflow.com/questions/29725932/deleting-rows-with-python-in-a-csv-file

import csv
import numpy as np

with open("..\\PixelFinder\\unsiftedEquatorPixelResults.csv", "r") as csv_file, open ("..\\PixelFinder\\unsiftedDunePixelResults.csv", "w") as out_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    writer = csv.writer(out_file)
    for row in csv_reader:
        if row[2] == "Dunes":
            writer.writerow(row)